In [7]:
import tkinter as tk
from tkinter.ttk import *
from PIL import ImageTk, Image
import webbrowser
import pandas as pd
import os

from tkinter import * 
import tkinter.messagebox
from tkinter import ttk
import tkinterweb
from IPython.display import Audio
from urllib.request import urlopen

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict, Counter
from tqdm import tqdm
from langdetect import detect, DetectorFactory
import multiprocessing
import os
import pickle

In [6]:
!pip install pillow
!pip install tk
!pip install sklearn
!pip install spotipy
!pip install numpy
!pip install langdetect
!pip install tqdm
!pip install pandas
!pip install tkinterweb



  Using cached tkinterweb-3.14.3.tar.gz (1.1 MB)
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for tkinterweb, since package 'wheel' is not installed.
  Running setup.py install for tkinterweb ... done


In [8]:
# Step 1 Initialize the system
# Load lyrics Data in, TFIDF vectorizer, LSI Object Creation, Lyric Vec dictionary, Open GUI, Spotify Cedentials.

# spotify credentials
cid = '3c72f1d18da74f63addd8423fd7d668f'
secret = '447a83ea7f494680a9bf88a43db64548'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)

token = util.prompt_for_user_token(
    username='12156878652',
    client_id = cid,
    client_secret = secret,
    redirect_uri = 'http://localhost:8000',
    scope=['user-modify-playback-state','user-read-playback-state']
    )
if token:
    sp = spotipy.Spotify(auth=token)

# convert the notebook into functions.
def load_lyrics_data():
    """
    return a dataframe with cleaned lyrics
    """
    temp_path = os.getcwd()
    root_path = temp_path.split('/napster_2')[0]
    repo_path = '/napster_2/search_functionality/final_merged_data.csv'
    data_path = root_path + repo_path
    # step 1 import old lyrical data into a dataframe.
    lyric_df = pd.read_csv(data_path)
    lyric_df = lyric_df[['track_name', 'artist_name', 'track_id','raw_lyrics']]
    # drop duplicate instances of the same track.
    lyric_df = lyric_df.drop_duplicates(subset='raw_lyrics').reset_index(drop=True)
    lyric_df = lyric_df.dropna()
    # replace new line character
    lyric_df['raw_lyrics'].replace('\n', ' ',regex=True, inplace=True)
    # remove embed text from lyric genius API
    lyric_df['raw_lyrics'].replace('[0-9]{1,3}Embed', '', regex=True, inplace=True)
    return lyric_df

def create_lyric_tfidf(lyric_df, min_df):
    """ 
    Create a tfidf vectorizer for the track lyrics
    """
    tfidf = TfidfVectorizer(tokenizer=str.split, min_df=min_df)
    # fit the TFIDF vectorizer
    tfidf.fit(lyric_df['raw_lyrics'])
    return tfidf

def lsi_lyrics(lyric_df, tfidf, num_concepts):
    """
    fit an LSI object using the lyrics
    """
    lyricTermMat = tfidf.transform(lyric_df['raw_lyrics'])
    lsiObj = TruncatedSVD(n_components=num_concepts, random_state=15)
    lsiObj.fit(lyricTermMat)
    return lsiObj

def create_lyric_vecs(lyric_df, lsiObj, tfidf):
    """ 
    generate the lyric vectors in the concept space
    """
    lyricTermMat = tfidf.transform(lyric_df['raw_lyrics'])
    lyric_vecs = lsiObj.transform(lyricTermMat)
    return lyric_vecs

def create_lyric_dictionary(lyric_df, lyric_vecs):
    """ 
    input a lyric dataframe and lsi lyric vectors
    return a dictionary where the track id is the key
    the vector is the value
    """
    # create a dataframe where the track id is the index the docVecs are the rows.
    track_vec_dict = defaultdict(list)
    track_ids = lyric_df['track_id'].values
    track_vec_dict = {track_ids[i]: lyric_vecs[i] for i in range(len(track_ids))}
    return track_vec_dict

def lsi_on_query(lsiObj, user_query, tfidf):
    """ 
    Transform the user search string into the concept space
    """
    userVec = tfidf.transform([user_query])
    # convert query vec into the concept space
    userLsi = lsiObj.transform(userVec)
    return userLsi

def retreive_10_tracks(lyric_df, userLsi, lyric_vecs):
    """ 
    calculate cosine similarity between lyrics and user query
    return the top 10 in a dataframe.
    """
    # calculate cosine similarity between every track and the lyric provided.
    simVals = cosine_similarity(lyric_vecs, userLsi)
    # create a track name, track id, artist name, similarity dataframe
    # need to make this a copy so that we do not modify the already stored data
    lyric_df['similarity'] = simVals
    user_playlist = lyric_df.sort_values(by='similarity', ascending=False)
    user_playlist = user_playlist.head(10)[['track_name', 'artist_name', 'track_id']]
    # initialize a feedback column and set every row to 0.
    user_playlist['feedback'] = 0
    return user_playlist

def simulate_user_input(user_playlist):
    """ 
    Simulate user input assign 0,1,2 to user feedback
    0 = nuetral
    1 = dislike
    2 = love
    """
    feedback = [np.random.randint(0,3) for i in range(len(user_playlist))]
    user_playlist['feedback'] = feedback
    return user_playlist

def rocchio_feedback(alpha, beta, gamma, phi, user_playlist, track_vec_dict, userLsi):
    """ 
    Rochhio Feedback Filtering
    1. Group user feeback by love, nuetral, dislike
    2. Calculate the mean for each group
    3. Apply alpha, beta, gamma, and phi to:
           Original search, loves, hates, nuetral
    4. Update the lyric search querry vector and return new results!
    return an updated query vector to improve search results
    """
    # create a mean vector dict for all 3 states
    meanVectDict = defaultdict(list)
    # iterate through the three states nuetral[0], dislike[1], love[2]
    for i in range(3):
        temp_tracks = user_playlist[user_playlist['feedback']==i]
        if len(temp_tracks) > 0:
            # this means that tracks with this sentiment exist.
            # we can go get the track vectors from the track_vec_dict
            tempVecs = [track_vec_dict[vec] for vec in temp_tracks['track_id']]
            # next we need to calculate the mean vector for this segment
            meanVec = np.mean(tempVecs, axis=0)
            # add this mean to the mean vect dict. The key is the state.
            meanVectDict[i] = meanVec
        else:
            # if there are no tracks in this state, set its mean to 0
            meanVectDict[i] = 0
    # calcualte the new query vector by summing all of the mean vectors together
    newQueryVec = alpha*userLsi + beta * meanVectDict[2] - gamma * meanVectDict[1] + phi * meanVectDict[0]
    return newQueryVec


#Action for clicking Get Lyric button
def reset():  
    global track_df
    global track_to_rate
    global like_dislike_counter
    
    sample_song_label.config(text="")
    #Once 10 songs have been rated, disable most buttons and allow user to view playlist
    if like_dislike_counter == 10:
        label.config(text="Track: \nArtist: ") #, font=("Arial",12))
        b_like["state"] = DISABLED
        b_dislike["state"] = DISABLED
        b_neutral["state"] = DISABLED
        b_listen["state"] = DISABLED
        b_playlist["state"] = NORMAL
    else:
        #display a random track for now
        track_to_rate = track_df.sample(n=1,replace=True) 
        #track_to_rate = track_df.iloc[like_dislike_counter] 
        label.config(text = 'Track: ' + track_to_rate.track_name_x.astype('str').item() + '\n' + 'Artist: ' + track_to_rate.artist_name_x.astype('str').item())

        if b_like["state"] == DISABLED:
            b_like["state"] = NORMAL
        if b_dislike["state"] == DISABLED:
            b_dislike["state"] = NORMAL
        if b_neutral["state"] == DISABLED:
            b_neutral["state"] = NORMAL
        if b_listen["state"] == DISABLED:
            b_listen["state"] = NORMAL
        if b_startover["state"] == DISABLED:
            b_startover["state"] = NORMAL
        #view_likes_dislikes()
    
# Add 1 to the rating counter and display progress to total songs needed to rate
def like_dislike_count():
    global like_dislike_counter
    like_dislike_counter += 1
    tracker_label = Label(frame1, text = 'Tracks Rated: ' + str("{0:0=2d}".format(like_dislike_counter)) + "/10")
    tracker_label.grid(row=2, column=0, sticky="nw", padx=5, pady = 10)

 
# Action for clicking like button: add to table at bottom of frame
def like():  
    global like_dislike_counter
    global listBox_lyrics
    global track_to_rate
    like_dislike_count()
    listBox_lyrics.insert("", "end", values=(track_to_rate.track_name_x.astype('str').item(), track_to_rate.artist_name_x.astype('str').item(), "Like")) 
    reset()
        
    
# Action for clicking Dislike button: add to table at bottom of frame  
def dislike():  
    global like_dislike_counter
    global listBox_lyrics
    global track_to_rate
    like_dislike_count()
    listBox_lyrics.insert("", "end", values=(track_to_rate.track_name_x.astype('str').item(), track_to_rate.artist_name_x.astype('str').item(), "Dislike")) 
    reset()
    
# Action for clicking Neutral button: add to table at bottom of frame  
def neutral():  
    global like_dislike_counter
    global listBox_lyrics
    global track_to_rate
    like_dislike_count()
    listBox_lyrics.insert("", "end", values=(track_to_rate.track_name_x.astype('str').item(), track_to_rate.artist_name_x.astype('str').item(), "Neutral")) 
    reset()
    
def listen_to_track():   
    global track_to_rate
    global sample_song_label
    track = track_to_rate.track_id.astype('str').item()
    track = sp.track(track)
    #print(track["preview_url"])
    
    try:
        #webbrowser.open_new(r"https://www.python.org")
        webbrowser.open_new(track["preview_url"])
        sample_song_label.config(text="Song preview playing in new window")
    except:
        sample_song_label.config(text="Song preview unavailable")
    
    '''
    frame_listen = tkinterweb.HtmlFrame(frame1)
    frame_listen.load_website(track["preview_url"]) 
    print(track["preview_url"])
    frame_listen.grid(row=1, column=1, pady=5, columnspan=3)
    '''
    #Audio(url="https://www.youtube.com/watch?v=5UMCrq-bBCg", autoplay=True)
 
# If you double click on a song in the display that was already rated, swap like / dislike
def enable_edit(event):
    curItem = listBox_lyrics.focus()
    curItem_values = listBox_lyrics.item(curItem)['values']
    if curItem_values[2] == 'Like':
        new_rating = 'Neutral'
    elif curItem_values[2] == 'Neutral':
        new_rating = 'Dislike'
    else:
        new_rating = 'Like'
    listBox_lyrics.item(curItem, values=(curItem_values[0], curItem_values[1],new_rating))
    for i in listBox_lyrics.selection():
        listBox_lyrics.selection_remove(i)
    
# Once enough songs are rated, the view playlist option displays and displays at bottom of frame
def view_playlist():
    # create table to show lyrics with rating
    global listBox_playlist
    global vsb_playlist
    
    # Hide lyric table
    #listBox_lyrics.grid_forget()
    
    vsb_playlist.grid(row=1, column=1, sticky='ns')
    listBox_playlist.grid(row=1, column=0, sticky="ew")
    
    #vsb.pack(side="right", fill="y") 
    #listBox_playlist.pack(side="left", fill="x", padx=5, pady=5)
    listBox_playlist.configure(yscrollcommand=vsb_playlist.set)
    
    #Insert songs from playlist dataframe
    for index, row in track_df.iterrows():
        listBox_playlist.insert("", "end", values=(row.track_name_x, row.artist_name_x))
    
    b_playlist["state"] = DISABLED
    b_export_playlist["state"] = NORMAL
    
# Reset button 
def start_over():
    global listBox_lyrics
    global listBox_playlist
    global like_dislike_counter
    
    # set counter back to 0 
    like_dislike_counter  = -1
    like_dislike_count()
    
    # Reset buttons
    b_playlist["state"] = DISABLED
    b_startover["state"] = DISABLED
    b_export_playlist["state"] = DISABLED
    
    # Remove lyric
    label.config(text="")
    
    # Reset table that displays playlist
    for item in listBox_playlist.get_children():
        listBox_playlist.delete(item)
    
    # Hide playlist and scrollbar
    listBox_playlist.grid_forget()
    vsb_playlist.grid_forget()
    
    # Reset table that displays liked / disliked lyrics
    for item in listBox_lyrics.get_children():
        listBox_lyrics.delete(item)
    
    listBox_lyrics.grid(row=1, column=0, sticky="ew")
    #listBox_lyrics.pack(side="left", fill=BOTH, padx=5, pady=5)
    lyricSearch["state"] = NORMAL
    reset()
    
def export_to_spotify():
    global sample_song_label
    for index, row in track_df.iterrows():
        try:
            sp.add_to_queue(row.track_id)
            sample_song_label.config(text="Songs have been added to your queue!")
        except:
            sample_song_label.config(text="Unable to add to playlist. Note: Spotify Premium required")
            break
    #sp.add_to_queue("13rC4iKtfQocWIfzPOJxaT")
    b_export_playlist["state"] = DISABLED

#Action for submitting lyric in search box
def lyric_processing():
    lyricText=lyricBox.get("1.0","end-1c")
    lyricSearch["state"] = DISABLED
    print(lyricText)     
    reset()


In [9]:
# global variables
# start with random lyric
#track_to_rate = track_df.sample(n=1,replace=True) #.track_id.astype('str').item()
like_dislike_counter = 0

# song lyric display
# creating tkinter window
root = Tk()
root.geometry('625x450')
root.title("Napster 2")

frame0 = Frame(root)
frame1 = Frame(root)
frame2 = Frame(root)
frame3 = Frame(root)

# Frame for get lyric, like, dislike
frame0.grid(row=0, column=0, sticky="w")
frame0.grid_columnconfigure((0,1,2,3), weight=1) #, uniform="column")

# Frame for get lyric, like, dislike
frame1.grid(row=1, column=0, sticky="w")
frame1.grid_columnconfigure((0,1,2,3), weight=1, uniform="column")

#Add buttons to sub-frame in Frame 1 so they all fit in first column
frame1b = Frame(frame1)
frame1b.grid(row=1, column=0, sticky="ew")
frame1b.grid_columnconfigure((0,1,2,3), weight=1, uniform="column")

# Frame to display like / dislike selections and playlist
frame2.grid(row=2, column=0, sticky="ew", rowspan=2) #, padx=3, pady=5)

# Display buttons to view playlist, start over, export playlist to spotify
frame3.grid(row=5, column=0, sticky="ew")
#frame3.grid_columnconfigure((0,1,2,3), weight=1, uniform="column")
#frame3.grid_columnconfigure((0,1,2,3), weight=1, uniform="column")


# Add label to be used for displaying lyric
#label = Label(frame1, text = 'Track: ' + track_to_rate.track_name_x.astype('str').item() + '\n' + 'Artist: ' + track_to_rate.artist_name_x.astype('str').item(), anchor='w', justify=LEFT) #, font=("Arial",12))
label = Label(frame1, text = 'Track: ' + '\n' + 'Artist: ' , anchor='w', justify=LEFT) #, font=("Arial",12))
label.grid(row=0, column=0, sticky="nw", padx=3, pady = 10, columnspan=4)

# Add images to buttons
temp_path = os.getcwd()
root_path = temp_path.split('/napster_2')[0]
tu_repo_path = '/napster_2/user_interface/thumbs_up.png'
td_repo_path = '/napster_2/user_interface/thumbs_down.png'
r_repo_path = '/napster_2/user_interface/refresh.png'
p_repo_path = '/napster_2/user_interface/play_button.png'
logo_repo_path = '/napster_2/user_interface/napster_logo.png'

tu_path = root_path + tu_repo_path
td_path = root_path + td_repo_path
r_path = root_path + r_repo_path
p_path = root_path + p_repo_path
logo_path = root_path + logo_repo_path

# Creating a photoimage object to use image
tu_photo = PhotoImage(file = tu_path)
td_photo = PhotoImage(file = td_path)
r_photo = PhotoImage(file = r_path)
p_photo = PhotoImage(file = p_path)
logo_photo = PhotoImage(file = logo_path)

# Resizing image to fit on button
tu_photoimage = tu_photo.subsample(6, 6)
td_photoimage = td_photo.subsample(6, 6)
r_photoimage = r_photo.subsample(25, 25)
p_photoimage = p_photo.subsample(12, 12)
logo_photoimage = logo_photo.subsample(2, 2)

#Add logo to top frame
logo_label = Label(frame0, image = logo_photoimage)
logo_label.grid(row=0, column=0)

#Add search bar
lyricBox = Text(frame0, height=2, width=32)
lyricBox.grid(row=0, column=1, pady=5)

lyricSearch = Button(frame0, text = 'Submit', command=lyric_processing)
lyricSearch.grid(row=0, column=2, sticky="ew", padx=3, pady=5)

# Create buttons (like, dislike, get lyrics, start over, view playlist)
b_like = Button(frame1b, image = tu_photoimage, compound = RIGHT, command=like)
b_like.grid(row=1, column=0, padx=3, pady=5)

b_dislike = Button(frame1b, image = td_photoimage, compound = RIGHT, command=dislike)
b_dislike.grid(row=1, column=1, padx=3, pady=5)

b_neutral = Button(frame1b, image = r_photoimage, compound = RIGHT, command=neutral)
b_neutral.grid(row=1, column=2, padx=2, pady=5)

b_listen = Button(frame1b, image = p_photoimage, compound = RIGHT, command=listen_to_track)
b_listen.grid(row=1, column=3, padx=2, pady=5)

# Track number of lyrics liked or disliked 
tracker_label = Label(frame1, text = 'Tracks Rated: ' + str("{0:0=2d}".format(like_dislike_counter)) + "/10")
tracker_label.grid(row=2, column=0, sticky="nw", padx=5, pady = 10)

# When play is clicked, display message on where song will play 
sample_song_label = Label(frame1, text = '')
sample_song_label.grid(row=1, column=1, sticky="nw", padx=5, pady = 10, columnspan=3)

# Start over button to get back to original screen
b_startover = Button(frame3, text = '        Start Over        ', command=start_over)
b_startover.grid(row=1, column=0, sticky="ew", padx=3, pady=5)

# View playlist once 10 lyrics rated
b_playlist = Button(frame3, text = '       View Playlist       ', command=view_playlist)
b_playlist.grid(row=1, column=1, sticky="ew", padx=3, pady=5)

# Export playlist to Spotify
b_export_playlist = Button(frame3, text = 'Add to Spotify Queue', command=export_to_spotify)
b_export_playlist.grid(row=1, column=2, sticky="ew", padx=3, pady=5)

# Store selection for likes and dislikes
#data = []    

# Set original state for like and dislike buttons to disables
b_like["state"] = DISABLED
b_dislike["state"] = DISABLED
b_neutral["state"] = DISABLED
b_listen["state"] = DISABLED
b_playlist["state"] = DISABLED
b_export_playlist["state"] = DISABLED

# create table to show lyrics with rating and add a scrollbar
cols = ('Track Name', "Artist Name", 'Rating')
listBox_lyrics = ttk.Treeview(frame2, columns=cols, show='headings')
vsb_lyrics = ttk.Scrollbar(frame2, orient="vertical", command=listBox_lyrics.yview)
#doubleclicking changes like to dislike and vice versa
listBox_lyrics.bind("<Double-1>", enable_edit)

for col in cols:
    listBox_lyrics.heading(col, text=col)    
#listBox_lyrics.grid(row=1, column=0, sticky="ew")

vsb_lyrics.grid(row=1, column=1, sticky='ns')
listBox_lyrics.grid(row=1, column=0, sticky="ew")
listBox_lyrics.configure(yscrollcommand=vsb_lyrics.set)

#listBox_lyrics.pack(side="left", fill=BOTH, padx=5, pady=5)

# create table to show playlist - do not display until enabled
cols = ("Track Name","Artist Name")
listBox_playlist = ttk.Treeview(frame2, selectmode='browse', columns=cols, show='headings')
vsb_playlist = ttk.Scrollbar(frame2, orient="vertical", command=listBox_playlist.yview)

for col in cols:
    listBox_playlist.heading(col, text=col)

mainloop()


Jealousy


Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.9/3.9.13_3/Frameworks/Python.framework/Versions/3.9/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/var/folders/6r/kwmwk0wx4yj2ccxtwqdqgb4r0000gn/T/ipykernel_22604/2844800702.py", line 326, in lyric_processing
    reset()
  File "/var/folders/6r/kwmwk0wx4yj2ccxtwqdqgb4r0000gn/T/ipykernel_22604/2844800702.py", line 163, in reset
    track_to_rate = track_df.sample(n=1,replace=True)
NameError: name 'track_df' is not defined


: 

: 